In [ ]:
import torch
import os
import numpy as np

# Model
from model.mlp import MyModel
# Data generation
from scripts.generation import Generator
# Plots
from scripts.plotter import Plotter
import matplotlib.pyplot as plt
# Movie generator:
import imageio.v2 as imageio

In [ ]:
# Set up the file paths
curpath = r'C:\Users\abelt\OneDrive\Dokumenter\GitHub\Ship_datafusion\playground'
folderpath = os.path.join(curpath, 'figures')
os.makedirs(folderpath, exist_ok=True)
video_storage = os.path.join(curpath,'movies')

# Clear folder for frames
def clear_folder(folder = folderpath):
    os.makedirs(folderpath, exist_ok=True)
    # Directory for saving frames, and cleaning it every run
    for item in os.listdir(folder):
        os.remove(os.path.join(folderpath, item))

# Forward with heading
def forward(t1, t2, model):
    e1 = model(torch.tensor(t1, dtype=torch.float32))
    e2 = model(torch.tensor(t2, dtype=torch.float32))
    return torch.cdist(e1,e2)


In [6]:
if __name__ == '__main__':
    # Set up model and optimizer
    maxiter = 1000
    N = 250
    N_test = 50
    sigma_distance = 0.025
    sigma_heading = 0.005
    encode = True
    missing_percentage = 0.6

    ranks = [1, 3, 5]

    mlp_params = {
        'input_dim': 4,
        'hidden_dim': 1024,
        'output_dim': 254,
        'n_layers': 5,
        'dropout': 0.2,
        'learning_rate': 0.00001,
    }

    clear_folder(folder=folderpath)

    # Initialize variables
    frames = []
    losses = {'train': [], 'val': []}

    # Create empty lists to store accuracies
    rank_accuracies = {rank: [] for rank in ranks}

    # Generate validation data
    p1, p2 = Generator.generate_data(N_test, sigma=sigma_distance)
    t1, t2 = Generator.calculate_heading(p1,p2, encoding=encode, missing_percentage=missing_percentage)
    
    model = MyModel(input_dim=mlp_params['input_dim'], hidden_dim=mlp_params['hidden_dim'], 
                output_dim=mlp_params['output_dim'], depth=mlp_params['n_layers'], 
                drop_prob=mlp_params['dropout'])
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=mlp_params['learning_rate'])
    # Training loop
    for i in range(maxiter + 1):
        model.train()
        optimizer.zero_grad()
        p1t, p2t = Generator.generate_data(N, sigma=sigma_distance)
        t1p, t2p = Generator.calculate_heading(p1t,p2t, encoding=encode, missing_percentage=missing_percentage)
        
        diag_matrix = forward(t1p, t2p, model)
        
        # Calculate training loss and backpropagate
        diag = torch.diag(diag_matrix)
        train_loss = torch.sum(diag)
        train_loss.backward()
        losses['train'].append(train_loss.item())
        optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            #diag_matrix_val = forward(p1, p2, model)
            diag_matrix_val = forward(t1, t2, model)

            diag_val = torch.diag(diag_matrix_val)
            val_loss = torch.sum(diag_val)
            losses['val'].append(val_loss.item())

        # Plot and save frame every 10 iterations
        if i % 10 == 0 or i == 1: # and i > 0
            # Rank-N evaluation
            for rank in ranks:
                accuracy = Generator.evaluate_rankN(diag_matrix, rank)
                rank_accuracies[rank].append(accuracy.item())

            d = torch.argmin(diag_matrix_val, dim=1).detach().numpy()

            # Plotting
            fig, (ax1, ax2, ax3) = Plotter.plot_iteration_with_loss_and_accuracy(
                t1, t2, diag_matrix_val, np.arange(N_test), losses, rank_accuracies, train_loss.item(), i, maxiter
            )
            frame_path = os.path.join(folderpath, f'frame_{i}.png')
            fig.savefig(frame_path)
            frames.append(imageio.imread(frame_path))
            plt.close(fig)

    # Save the video
    imageio.mimsave(os.path.join(folderpath, '0_training_epochs.mp4'), frames, fps=3)